# Analyze based on semantic categories

1.) change tfidf so we compare equivalent categories only - done
2.) update ranking accordingly

In [1]:
import os
from collections import Counter, defaultdict
import csv
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)


def get_properties():
    properties = []
    for path in os.listdir('../data/aggregated/'):
        prop = path.split('.')[0]
        if 'female-' not in prop and prop != '':
            properties.append(prop)
    return properties

In [3]:
# check updated corpus extraction
f_original = os.listdir('../contexts/giga_full/vocab')
print(len(f_original))
f_update = os.listdir('../contexts/giga_full_updated/vocab')
print(len(f_update))

1446
1636


## Show top distinctive contexts per property

* calculate mean f1 per sematic category
* rank contexts by mean f1
* only include categories with at least 10 pos and 10 neg examples (small categories distort mean)

In [9]:
pd.set_option('display.max_colwidth', None)
# get top distinctive contexts per prop
import get_top_contexts_distinctiveness

model_name = 'giga_full_updated'
properties = get_properties()
table = get_top_contexts_distinctiveness.get_top_distinctive_contexts(properties, model_name)
df = pd.DataFrame(table)
df.sort_values('f1-mean', ascending = False).round(2)

,property,n_contexts,f1-mean,contexts
9,used_in_cooking,572,0.94,add recipe fish
4,fly,952,0.90,payload study experimental hovering flew overhead
11,green,578,0.86,place shade belongs green citrus
0,square,1504,0.85,built
13,blue,2234,0.85,magic bright scale various european
16,female,45,0.84,actress birth herself
20,lay_eggs,74,0.83,eggs
8,hot,84,0.83,flame oven remove hot
7,sweet,35,0.83,banana potato sweet
14,yellow,52,0.83,sipping operating apple yellow good bubbly apples


In [6]:
# latex table for paper:
cols = ['property', 'n_contexts', 'f1-mean', 'contexts']
df = df.sort_values('f1-mean', ascending = False).round(2)
print(df[cols].to_latex(index=False))

\begin{tabular}{lrrl}
\toprule
        property &  n\_contexts &  f1-mean &                                           contexts \\
\midrule
 used\_in\_cooking &         572 &     0.94 &                                    add recipe fish \\
             fly &         952 &     0.90 &  payload study experimental hovering flew overhead \\
           green &         578 &     0.86 &                   place shade belongs green citrus \\
          square &        1504 &     0.85 &                                              built \\
            blue &        2234 &     0.85 &                magic bright scale various european \\
          female &          45 &     0.84 &                              actress birth herself \\
        lay\_eggs &          74 &     0.83 &                                               eggs \\
             hot &          84 &     0.83 &                              flame oven remove hot \\
           sweet &          35 &     0.83 &                               

In [10]:
# get top distinctive contexts per prop

model_name = 'wiki_updated'
properties = get_properties()
table = get_top_contexts_distinctiveness.get_top_distinctive_contexts(properties, model_name)
df = pd.DataFrame(table)
df.sort_values('f1-mean', ascending = False).round(2)

,property,n_contexts,f1-mean,contexts
9,used_in_cooking,705,0.92,meat food
16,female,109,0.90,la female birth she
4,fly,55,0.90,thrust fly demonstrate bird experimental flown owl
14,yellow,111,0.88,name whiskey beer tribe half
6,wings,332,0.87,birds
13,blue,1819,0.87,simple abundant evening
2,black,820,0.86,north any size black accidentally typical led
19,wheels,105,0.85,chassis driver
11,green,550,0.85,fruits flora green found citrus forest
5,dangerous,627,0.84,hand killing


In [8]:
# latex table for paper:
cols = ['property', 'n_contexts', 'f1-mean', 'contexts']
df = df.sort_values('f1-mean', ascending = False).round(2)
print(df[cols].to_latex(index=False))

\begin{tabular}{lrrl}
\toprule
        property &  n\_contexts &  f1-mean &                                           contexts \\
\midrule
 used\_in\_cooking &         705 &     0.92 &                                          meat food \\
          female &         109 &     0.90 &                                la female birth she \\
             fly &          55 &     0.90 &  thrust fly demonstrate bird experimental flown... \\
          yellow &         111 &     0.88 &                       name whiskey beer tribe half \\
           wings &         332 &     0.87 &                                              birds \\
            blue &        1819 &     0.87 &                            simple abundant evening \\
           black &         820 &     0.86 &      north any size black accidentally typical led \\
          wheels &         105 &     0.85 &                                     chassis driver \\
           green &         550 &     0.85 &             fruits flora green 

## Prepare context annotation

In [ ]:
# import prepare_annotations

# model_name = 'giga_full_updated'
# properties = get_properties()
# #properties_test = ['dangerous', 'cold', 'lay_eggs']
# #properties = [p for p in properties if p not in properties_test]
# #properties = properties_test
# properties = ['dangerous']
# cutoff = 3
# cutoff_concepts = 3

# for prop in properties:
#     print(prop)
    
#     prepare_annotations.aggregate_contexts(prop, cutoff, model_name)
#     prepare_annotations.prepare_annotation(prop, model_name, cutoff, cutoff_concepts)

### Transfer giga to wiki annotations 


In [11]:
# run: python transfer_annotations.py

### Complete annotations

In [2]:
import analyze_annotations
import numpy as np
import pandas as pd

In [5]:
model_name = 'wiki_updated'
top_cutoff = 3
concept_cutoff = 3
ann_dict = analyze_annotations.show_annotation_status(model_name, top_cutoff, concept_cutoff)

completed:

black (821, 634, 187)
blue (1820, 1018, 802)
cold (591, 382, 209)
dangerous (628, 392, 236)
female (110, 97, 13)
fly (56, 41, 15)
green (551, 402, 149)
hot (93, 78, 15)
juicy (123, 59, 64)
lay_eggs (27, 18, 9)
made_of_wood (498, 373, 125)
red (1276, 781, 495)
roll (3486, 1464, 2022)
round (1035, 886, 149)
square (443, 214, 229)
sweet (27, 20, 7)
swim (1701, 1225, 476)
used_in_cooking (706, 492, 214)
warm (1182, 573, 609)
wheels (106, 66, 40)
wings (333, 234, 99)
yellow (112, 108, 4)

Incomplete:



### Consistency checks

1.) Consistency between the corpora - done

2.) Property words - done

3.) Instances that are positive examples - done

4.) Sample instances and let them be double-annotated?

In [37]:
def get_agreement_corpora(prop, model_name1, model_name2, top_cutoff, concept_cutoff):
    table = []
    ann_dict1 = analyze_annotations.get_evidence_dict(model_name1, prop, top_cutoff, concept_cutoff)
    ann_dict2 = analyze_annotations.get_evidence_dict(model_name2, prop, top_cutoff, concept_cutoff)

    all_words = set(ann_dict1.keys()).union(ann_dict2.keys())

    for w in all_words:
        wd = dict()
        wd['context'] = w
        if w in ann_dict1:
            l1 = ann_dict1[w]
        else:
            l1 = 'nic'
        if w in ann_dict2:
            l2 = ann_dict2[w]
        else:
            l2 = 'nic'
        wd[model_name1] = l1
        wd[model_name2] = l2
        wd['agreement'] = True
        wd['overlap'] = False
        if 'nic' not in [l1, l2]:
            wd['overlap'] = True
            if l1 != l2:
                wd['agreement'] = False
        else:
            wd['agreement'] = '-'
        table.append(wd)
    return table


def check_for_property_words(model_name, top_cutoff, concept_cutoff):
    
    prop_word_dict = {
        'lay_eggs': {'egg',  'eggs', 'lay', 'laying', 'laid', 'lays'},
        'used_in_cooking': {'cook', 'cooked', 'cooks', 'cooking'},
        'cold': {'cold', 'cool', 'colder', 'coldest'},
        'warm' : {'warm', 'warmer', 'warmest'},
        'sweet': {'sweeter', 'sweetest'},
        'made_of_wood': {'wood', 'wooden'},
        'wings': {'wing', 'wings'},
        'swim': {'swim', 'swam', 'swimming', 'swims'},
        'fly': {'fly', 'flies', 'flew', 'flying', 'flown'},
        'roll': {'roll', 'rolled', 'rolling', 'rolls'},
        'hot': {'hot', 'hotter', 'hottest'},
        'wing': {'wings', 'wing', 'winged'},
        'wheels': {'wheel',  'wheels', 'wheeled'},
        
    }
    
    properties = get_properties()
    properties.append('female')
    properties = set(properties)
    
    for prop in properties:
        ann_dict = analyze_annotations.get_evidence_dict(model_name, prop, top_cutoff, concept_cutoff)
        if prop in prop_word_dict:
            prop_words = prop_word_dict[prop]
        else:
            prop_words = {prop}
        for w, l in ann_dict.items():
            if w in prop_words and l != 'p':
                print(prop, w, l)
        
    
    
    

In [108]:
model_name1 = 'wiki_updated'
model_name2 = 'giga_full_updated'
top_cutoff = 3
concept_cutoff = 3

properties = get_properties()


for prop in properties:
    table = get_agreement_corpora(prop, model_name1, model_name2, top_cutoff, concept_cutoff)
    df = pd.DataFrame(table)
    df_short = df[df['agreement'] == False]
    print(prop, len(df_short))
    if len(df_short) > 0:
        break
    

square 0
warm 0
black 0
red 0
fly 0
dangerous 0
wings 0
sweet 0
hot 0
used_in_cooking 0
juicy 0
green 0
made_of_wood 0
blue 0
yellow 0
roll 0
female 0
cold 0
round 0
wheels 0
lay_eggs 0
swim 0


In [109]:
df_short

,context,wiki_updated,giga_full_updated,agreement,overlap


In [44]:
model_name = 'wiki_updated'
top_cutoff = 3
concept_cutoff = 3
check_for_property_words(model_name, top_cutoff, concept_cutoff)

In [45]:
model_name = 'giga_full_updated'
top_cutoff = 3
concept_cutoff = 3
check_for_property_words(model_name, top_cutoff, concept_cutoff)

cold cool n


In [101]:
# instances
import analyze_evidence


model_name = 'wiki_updated'
top_cutoff = 3
concept_cutoff = 3

properties = set(get_properties())
properties.add('female')


for prop in properties:
    prop_data = analyze_evidence.load_prop_data(prop)
    pos_labels = ['all', 'all-some', 'some', 'few-some']
    pos_concepts = [c for c, d in prop_data.items() if d['ml_label'] in pos_labels]


    # check annotations
    ann_dict = analyze_annotations.get_evidence_dict(model_name, prop, top_cutoff, concept_cutoff)

    for w, l in ann_dict.items():
        if w in pos_concepts and l != 'i':
            print(prop, w, l)

hot charcoal r
hot heat p
round chocolate u
juicy juice p
made_of_wood bamboo p
made_of_wood timber p
made_of_wood wood p
made_of_wood oak p
made_of_wood pine p
warm smoke r
warm winter r
swim bay r
roll wheel l
roll plastic u
roll cloth u
roll axle l
roll paint u
roll pig u
roll rail r
roll bearing l
roll saw u
wheels wheel p


### Evidence density

*How many of the evidence candidates are actual property evidence? How are they distributed over differnt types of evidence?*

In [39]:
model_name = 'giga_full_updated'
top_cutoff = 3
concept_cutoff = 3
ann_dict = analyze_annotations.show_annotation_status(model_name, top_cutoff, concept_cutoff)
properties = ann_dict['complete']


cols = ['property', 'u', 'all', 'prop_specific', 'non-specific', 'p', 'n', 'l', 'i', 'r', 'b']
table = []
for prop in properties:
    d = dict()
    d['property'] =  prop
    d.update(analyze_annotations.get_evidence_density(model_name, prop, top_cutoff, concept_cutoff))
    for c in cols:
        if c not in d:
            d[c] = np.nan
    table.append(d)
   
cols = ['property', 'u', 'all', 'prop_specific', 'non-specific', 'p', 'n', 'l', 'i', 'r', 'b']
df = pd.DataFrame(table)[cols]
df = df[cols].sort_values('all', ascending = False)
median = df.median(axis=0)
df.loc['median'] = median

df.round(3)
print(df.round(3).fillna('-').to_latex(index=False))

completed:

black (1150, 0, 1150)
blue (2235, 0, 2235)
cold (1082, 0, 1082)
dangerous (1115, 0, 1115)
female (46, 0, 46)
fly (953, 0, 953)
green (579, 0, 579)
hot (85, 0, 85)
juicy (811, 0, 811)
lay_eggs (75, 0, 75)
made_of_wood (787, 0, 787)
red (1768, 0, 1768)
roll (3887, 0, 3887)
round (521, 0, 521)
square (1505, 0, 1505)
sweet (36, 0, 36)
swim (1382, 0, 1382)
used_in_cooking (573, 0, 573)
warm (2109, 0, 2109)
wheels (245, 0, 245)
wings (561, 0, 561)
yellow (53, 0, 53)

Incomplete:

\begin{tabular}{lrrlrlllrll}
\toprule
        property &      u &    all & prop\_specific &  non-specific &      p &      n &      l &      i &      r &      b \\
\midrule
 used\_in\_cooking &  0.355 &  0.645 &         0.014 &         0.631 &  0.007 &  0.003 &  0.003 &  0.329 &  0.302 &      - \\
           sweet &  0.400 &  0.600 &         0.057 &         0.543 &  0.057 &      - &      - &  0.543 &      - &      - \\
          female &  0.622 &  0.378 &         0.067 &         0.311 &      - &      - & 

In [6]:
#

### Evidence diversity and coherence

*How many different words are used to express property evidence?*

In [44]:
# run: python get_diversity_coherence.py (change model name and path)

# load results:

model_name = 'wiki_updated'

path_table = f'../analysis/diversity-coherence-{model_name}.csv'
path_table_median = f'../analysis/diversity-coherence-{model_name}-median.csv'
df = pd.read_csv(path_table)#.round(2)
df.drop(['Unnamed: 0'], axis=1, inplace=True)

median = df.median(axis=0)
df.loc['median'] = median

df = df.round(2)
df = df.fillna('-')
df.to_csv(path_table_median)

print(df.to_latex(index=False))
#df

\begin{tabular}{lrrrlrr}
\toprule
        property &  all-div &  all-sim &  prop\_specific-div & prop\_specific-sim &  non-specific-div &  non-specific-sim \\
\midrule
            roll &    130.0 &     0.15 &               10.0 &              0.26 &             120.0 &              0.15 \\
    made\_of\_wood &     43.0 &     0.17 &                9.0 &              0.38 &              34.0 &              0.15 \\
 used\_in\_cooking &    268.0 &     0.19 &                9.0 &              0.41 &             259.0 &              0.19 \\
             hot &     25.0 &     0.21 &                8.0 &              0.23 &              17.0 &              0.21 \\
            warm &     87.0 &     0.14 &                7.0 &              0.34 &              80.0 &              0.13 \\
           wings &     57.0 &     0.20 &                6.0 &              0.54 &              51.0 &              0.19 \\
       dangerous &     75.0 &     0.17 &                6.0 &              0.28 &         

### Evidence distinctiveness

In [3]:
# import json
# import csv
# import os
# from collections import Counter, defaultdict
import analyze_evidence

import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [5]:
prop = 'juicy'
model_name = 'giga_full_updated'
top_cutoff = 3
concept_cutoff = 3
table = analyze_evidence.get_top_ev_categories(prop, model_name, top_cutoff, concept_cutoff)
table_mean = analyze_evidence.get_mean_ev_categories(prop, model_name, top_cutoff, concept_cutoff)
for k, d in table.items():
    d['f1_max'] = d['f1']
    d.pop('f1')
    d_mean = table_mean[k]
    d['f1_mean'] = d_mean['f1']

cols = ['f1_mean', 'f1_max', 'contexts', 'pos', 'neg']
df = pd.DataFrame(table)
df = df.T[cols]
df

f1_mean f1_max      contexts pos neg
vegetable prop_specific     0.7   0.79        juices  15  14
          non_specific     0.52   0.82         fruit  15  14
          p                 0.7   0.79        juices  15  14
          i                0.52   0.82         fruit  15  14
          r                0.53   0.76         fresh  15  14
          u                0.52    0.9           for  15  14
plant     prop_specific    0.73   0.73  juices juice  59  28
          non_specific     0.44    0.8          ripe  59  28
          p                0.73   0.73  juices juice  59  28
          i                0.43   0.79     pineapple  59  28
          r                0.48    0.8          ripe  59  28
          u                0.44    0.8        banana  59  28
fruit     prop_specific     0.8    0.8         juice  53  25
          non_specific     0.54   0.85     pineapple  53  25
          p                 0.8    0.8         juice  53  25
          i                0.53   0.85     pineapple  53  25
          r                0.66   0.81         fresh  53  25
          u                0.43   0.81           red  53  25
object    prop_specific    0.76   0.79        juices  73  50
          non_specific     0.53   0.81     pineapple  73  50
          p                0.76   0.79        juices  73  50
          i                0.52   0.81     pineapple  73  50
          r                0.58   0.78          ripe  73  50
          u                0.48   0.76        bright  73  50
food      prop_specific    0.69   0.71        juices  83  41
          non_specific     0.49   0.79     pineapple  83  41
          p                0.69   0.71        juices  83  41
          i                0.48   0.79     pineapple  83  41
          r                0.52   0.74          ripe  83  41
          u                0.46   0.76        bright  83  41
all       prop_specific    0.72   0.74        juices  88  60
          non_specific     0.52    0.8     pineapple  88  60
          p                0.72   0.74        juices  88  60
          i                0.51    0.8     pineapple  88  60
          r                0.55   0.73          ripe  88  60
          u                0.47   0.73           red  88  60

In [6]:
#print(df.to_latex())

In [3]:
#

In [13]:
model_name = 'wiki_updated'
top_cutoff = 3
concept_cutoff = 3

pd.options.display.float_format = "{:,.2f}".format

properties = get_properties()
prop_table = []
for prop in properties:
    table = analyze_evidence.get_top_ev_categories(prop, model_name, top_cutoff, concept_cutoff)
    table_mean = analyze_evidence.get_mean_ev_categories(prop, model_name, top_cutoff, concept_cutoff)
    for k, d in table.items():
        d['f1_max'] = d['f1']
        d.pop('f1')
        d_mean = table_mean[k]
        d['f1_mean'] = d_mean['f1']
    type_scores = defaultdict(list)
    ev_type_mean_scores = dict()
    for (cat, ev_type), f1_dict in table.items():
        type_scores[f'{ev_type}-mean'].append(f1_dict['f1_mean'])
        type_scores[f'{ev_type}-max'].append(f1_dict['f1_max'])

    for ev_type, scores in type_scores.items():
        mean = sum(scores)/len(scores)
        prop_dict = dict()
        ev_type_mean_scores[ev_type] = mean
    prop_dict['property'] = prop
    prop_dict.update(ev_type_mean_scores)
    prop_table.append(prop_dict)
    
#cols = ['property','prop_specific', 'non_specific', 'p', 'n', 'l', 'i', 'r', 'b', 'u']
cols = ['property','prop_specific', 'non_specific', 'u']
cols_full = ['property']
for col in cols[1:]:
    for sc in ['mean', 'max']:
        cols_full.append(f'{col}-{sc}')

df = pd.DataFrame(prop_table)
df = df[cols_full].sort_values('prop_specific-mean', ascending = False)
median = df.median(axis=0)
df.loc['median'] = median
df = df.fillna('-')#round(2).fillna('-')
df
print(df.to_latex(index=False))

\begin{tabular}{lllrrrr}
\toprule
        property & prop\_specific-mean & prop\_specific-max &  non\_specific-mean &  non\_specific-max &  u-mean &  u-max \\
\midrule
            blue &               0.84 &              0.84 &               0.64 &              0.85 &    0.62 &   0.87 \\
           green &               0.81 &              0.81 &               0.52 &              0.82 &    0.46 &   0.81 \\
             red &               0.76 &              0.76 &               0.47 &              0.73 &    0.47 &   0.76 \\
             fly &               0.74 &              0.83 &               0.72 &              0.89 &    0.59 &   0.80 \\
           juicy &               0.73 &              0.73 &               0.59 &              0.82 &    0.50 &   0.76 \\
       dangerous &               0.72 &              0.81 &               0.58 &              0.81 &    0.50 &   0.80 \\
          female &               0.72 &              0.80 &               0.70 &              0.88 &    0.

In [19]:
#print(df.to_latex(index=False))

## Evidence strength


In [9]:
import analyze_evidence


def get_tfidf_scores_context(prop, cat, contexts, model_name):
    
    # collect scores here:
    context_tfidf_dict = defaultdict(dict)

    # get tfidf scores
    path_tfidfs = f'../results/{model_name}/tfidf-raw-10000/each_target_vs_corpus_per_category'
    paths_tfidf = [f'{path_tfidfs}/{prop}/{cat}/pos', f'{path_tfidfs}/{prop}/{cat}/neg']
    concept_files = []
    for path in paths_tfidf:
        concept_files.extend([f'{path}/{f}' for f in os.listdir(path) if f.endswith('.csv')])
    for cf in concept_files:
        full_path = cf
        #full_path = f'{path_tfidfs}/{cf}'
        concept = os.path.basename(cf).split('.')[0]
        with open(full_path) as infile:
            data = list(csv.DictReader(infile))
        for cw_target in contexts:
            found = False
            for d in data:
                cw = d['']
                score = d['target']
                if cw == cw_target:
                    context_tfidf_dict[cw][concept] = (float(score))
                    found = True
                    break
            if found == False:
                context_tfidf_dict[cw_target][concept]  = 0.0
    return context_tfidf_dict


def evidence_strength_to_file(prop, et, contexts, model_name, top_cutoff, concept_cutoff):

    filepath_target = f'../analysis/{model_name}/evidence_{top_cutoff}_{concept_cutoff}-raw-10000-categories'
    filepath_target_et = f'{filepath_target}/{prop}/{et}'
    os.makedirs(filepath_target_et, exist_ok=True)
    categories = analyze_evidence.get_categories(prop, model_name)
    tfidf_scores_cats = dict()
              
    for context in contexts:
        full_filepath = f'{filepath_target_et}/{context}.csv'
        for cat in categories:
            tfidf_scores_cats[cat] = get_tfidf_scores_context(prop, cat, contexts, model_name)[context]
            #print(cat, tfidf_scores_cats[cat]['wren'])
        df = pd.DataFrame(tfidf_scores_cats)
        df.fillna(0, inplace=True)
        df['mean'] = df.mean(numeric_only=True, axis = 1)
        df.to_csv(full_filepath)

In [11]:

# extract mean evidence strength for context words that give prop-specific evidence
model_name = 'giga_full_updated'
properties = get_properties()

#properties = ['dangerous']

top_cutoff = 3
concept_cutoff = 3


for prop in properties:
    print(prop)
    table = analyze_evidence.get_top_ev_categories(prop, model_name, top_cutoff, concept_cutoff)

    ets = ['p', 'n', 'l']
    for et_target in ets:
        contexts = set()
        for (cat, et), d in table.items():
            if et_target == et:
                contexts.update(set(d['contexts'].split(' ')))
        if len(contexts) > 0:
            evidence_strength_to_file(prop, et_target, contexts, model_name, top_cutoff, concept_cutoff)

square
warm
black
red
fly
dangerous
wings
sweet
hot
used_in_cooking
juicy
green
made_of_wood
blue
yellow
roll
female
cold
round
wheels
lay_eggs
swim


In [79]:
prop_dict = analyze_evidence.load_prop_data(prop)

#prop_dict

In [81]:
# compare evidence strength across properties - may not make too much sense...

ets = ['p', 'l', 'n']
properties = get_properties()

model_name = 'giga_full_updated'

top_cutoff = 3
concept_cutoff = 3

prop_et_strength_dict = defaultdict(dict)

path_evidence = f'../analysis/{model_name}/evidence_{top_cutoff}_{concept_cutoff}-raw-10000-categories'

for prop in properties:
    means_prop = []
    prop_dict = analyze_evidence.load_prop_data(prop)
    # only use pos examples:
    pos_ml_labels = ['all', 'all-some', 'few-some', 'some']
    concepts_pos = [c for c, d in prop_dict.items() if d['ml_label'] in pos_ml_labels]
    for et in ets:
        means_et = []
        path_dir = f'{path_evidence}/{prop}/{et}'
        if os.path.isdir(path_dir):
            ev_files = [f for f in os.listdir(path_dir) if f.endswith('.csv')]
            for f in ev_files:
                full_path = f'{path_dir}/{f}'
                with open(full_path) as infile:
                    data = list(csv.DictReader(infile))
                    means = [float(d['mean']) for d in data if d[''] in concepts_pos]
                    if len(means) > 0:
                        mean = sum(means)/len(means)
                    else:
                        mean = 0.0
                    #means_et.append(mean)
                    means_prop.append(mean)
    if len(means_prop) > 0:
        mean_prop = sum(means_prop)/len(means_prop)
    else:
        mean_prop = 0.0
    prop_et_strength_dict[prop]['prop_specific'] = mean_prop
            
df = pd.DataFrame(prop_et_strength_dict)
df.T.sort_values('prop_specific', ascending=False).round(4)

,prop_specific
juicy,0.0188
female,0.0168
lay_eggs,0.0138
red,0.0079
sweet,0.0070
hot,0.0069
yellow,0.0065
made_of_wood,0.0064
wheels,0.0060
green,0.0060


## Relation analysis

In [40]:
from statistics import stdev
import numpy as np
import analyze_evidence
pd.set_option('display.max_colwidth', None) 

In [62]:

  
def get_relation_combinations(properties, combinations, rel_select = 'strict'):
    
    relation_pair_dict = defaultdict(set)
    for prop in properties:
        prop_dict = analyze_evidence.load_prop_data(prop)
        for c, d in prop_dict.items():
            ml_label = d['ml_label']
            if ml_label in {'all', 'some', 'all-some', 'few-some'}:
                l = 'pos'
            elif ml_label in {'few'}:
                l = 'neg' 
            relation_pair_dict[l].add((prop, c))
            if ml_label in {'all', 'all-some'}:
                l_qu = 'all'
            elif ml_label in {'some', 'few-some'}:
                l_qu = 'some'
            elif ml_label in {'few'}:
                l_qu = 'few'
            relation_pair_dict[l_qu].add((prop, c))
            rel_dict = d['relations']
            for combination in combinations:
                if rel_select == 'strict':
                    relations = set([rel for rel, p in rel_dict.items() if p > 0.5])
                elif type(rel_select) == tuple:
                    upper, min_dist = rel_select
                    # sort relations:
                    prop_rel_dict = defaultdict(list)
                    for rel, p in rel_dict.items():
                        prop_rel_dict[p].append(rel)
                    pos_props = sorted(list(prop_rel_dict.keys()), reverse=True)
                    top = pos_props[0]
                    if len(pos_props) > 1:
                        second = pos_props[1]
                    else:
                        second = 0.0
                    dist = top - second
                    if top > upper and dist > min_dist:
                        relations = set(prop_rel_dict[top])
                    else:
                        relations = set()
                if combination == relations:
                    l_comb = tuple(sorted(relations))
                    relation_pair_dict[l_comb].add((prop, c))
    return relation_pair_dict   

In [43]:

# get prop concept pairs with a certain relation-configuration:
combinations = [
                    {'implied_category'},
                    {'variability_limited'},
                    {'variability_open'},
                    {'afforded_usual'},
                    {'afforded_usual', 'implied_category'},
                    {'afforded_unusual'},
                    {'implied_category', 'variability_limited'},
                    {'typical_of_property'}, 
                    {'typical_of_concept'},
                    {'typical_of_concept', 'typical_of_property'}
                    ]

properties = get_properties()
properties = [p for p in properties if p != 'female']
relation_pair_dict = get_relation_combinations(properties, combinations)
for rel, pairs in relation_pair_dict.items():
     print(rel, len(pairs)) 
# print()
# print(relation_pair_dict[('implied_category',  )]) 
# print()
# print(relation_pair_dict[('variability_limited',  )]) 
# print()
# print(relation_pair_dict[('implied_category', 'variability_limited', )])

print()
print(relation_pair_dict[('typical_of_concept',  )]) 
print()
print(relation_pair_dict[('typical_of_concept', 'typical_of_property' )]) 
print()
# print(relation_pair_dict[('implied_category', 'variability_limited', )])
# print()
# print(relation_pair_dict[('variability_limited', )])


neg 1331
few 1331
pos 1973
some 829
all 1144
('implied_category', 'variability_limited') 21
('variability_open',) 175
('variability_limited',) 114
('implied_category',) 16
('typical_of_concept', 'typical_of_property') 4
('typical_of_concept',) 4
('afforded_unusual',) 20
('afforded_usual', 'implied_category') 18
('afforded_usual',) 3

{('made_of_wood', 'club'), ('juicy', 'meat'), ('fly', 'babbler'), ('wheels', 'wheel')}

{('green', 'jade'), ('red', 'watermelon'), ('red', 'tomato'), ('sweet', 'desert')}



In [44]:
# focus on prop-specific evidence types


def get_tfidf_pair(ets, prop, concept, model_name, top_cutoff, concept_cutoff):
    
    path_evidence = f'../analysis/{model_name}/evidence_{top_cutoff}_{concept_cutoff}-raw-10000-categories'
    tfidf_scores = []
    for et in ets:
        path_prop_et = f'{path_evidence}/{prop}/{et}'
        if os.path.isdir(path_prop_et):
            files = [f for f in os.listdir(path_prop_et) if f.endswith('.csv')]
            for f in files:
                full_path = f'{path_prop_et}/{f}'
                with open(full_path) as infile:
                    data = list(csv.DictReader(infile))
                for d in data:
                    if d[''] == concept:
                        tfidf = d['mean']
                        tfidf_scores.append(float(tfidf))
                        break
    if len(tfidf_scores) > 0:
        mean = sum(tfidf_scores)/len(tfidf_scores)
    else:
        mean = 0.0
    return mean
            

#### Sanity check: 

Positive relations should have a higher mean tfidf value than negative relations

In [45]:
model_name  = 'giga_full_updated'
top_cutoff = 3
concept_cutoff = 3

ets = ['p', 'l', 'n']
combination = ('pos')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(ets, prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

1973
0.004043487156220856


In [46]:
combination = ('neg')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(ets, prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

1331
0.0009794759177106072


### Step 1: Check isolated relations

* advantage: no interference from other relations
* disadvantage: small number of prop-concept pairs

### Step 2: Check effects of combinations

* impliedness + relation associated with prop mentions should lead to higher values
* typical_of_concept + relation associated with prop mentions should lead to higher values

In [63]:
# overview all relations
rel_table = []

all_relations = ['pos', 'neg', 'all', 'some', 'few',
                 'implied_category', 
                 'typical_of_concept', 'typical_of_property', 
                 'affording_activity', 'afforded_usual', 'afforded_unusual',
                 'variability_limited', 'variability_open',
                 'rare', 'unusual', 'impossible', 'creative']

for rel in all_relations:
    rel_dict = dict()
    rel_dict['relation'] = rel
    if rel in ['pos', 'neg', 'all', 'some', 'few']:
        combination = (rel)
    else:
        combination = (rel, )
    combinations = [set(combination)]
    relation_pair_dict = get_relation_combinations(properties, combinations, rel_select = 'strict')
    pairs = relation_pair_dict[combination]
    n_pairs = len(pairs)
    rel_dict
    means = []
    for prop, concept in pairs:
        means.append(get_tfidf_pair(ets, prop, concept, model_name, top_cutoff, concept_cutoff))
    if len(means) > 0:
        rel_dict['mean_tfidf'] = sum(means)/len(means)
    else:
        rel_dict['mean_tfidf'] = 0
    rel_dict['n_pairs'] = n_pairs
    if n_pairs > 3:
        n_max = 3
    else:
        n_max = n_pairs
    rel_dict['examples'] = ' '.join(['-'.join(pair) for pair in list(pairs)[:n_max]])
    rel_table.append(rel_dict)

df = pd.DataFrame(rel_table)
df.round(6)

,relation,mean_tfidf,n_pairs,examples
0,pos,0.004043,1973,used_in_cooking-pestle red-pot green-pear
1,neg,0.000979,1331,roll-windshield blue-crow lay_eggs-mangabey
2,all,0.004780,1144,used_in_cooking-pestle used_in_cooking-parer square-pantry
3,some,0.003027,829,blue-fish red-pot square-macintosh
4,few,0.000979,1331,roll-windshield blue-crow lay_eggs-mangabey
5,implied_category,0.001294,16,warm-brogue wheels-tank swim-bay
6,typical_of_concept,0.007615,4,made_of_wood-club juicy-meat fly-babbler
7,typical_of_property,0.000000,0,
8,affording_activity,0.006053,4,wheels-rig juicy-okra sweet-allspice
9,afforded_usual,0.009278,3,swim-goldeneye lay_eggs-platypus lay_eggs-stockfish


In [75]:
# overview all relations
rel_table = []

all_relations = ['pos', 'neg', 'all', 'some', 'few',
                 'implied_category', 
                 'typical_of_concept', 'typical_of_property', 
                 'affording_activity', 'afforded_usual', 'afforded_unusual',
                 'variability_limited', 'variability_open',
                 'rare', 'unusual', 'impossible', 'creative']

for rel in all_relations:
    rel_dict = dict()
    rel_dict['relation'] = rel
    if rel in ['pos', 'neg', 'all', 'some', 'few']:
        combination = (rel)
    else:
        combination = (rel, )
    combinations = [set(combination)]
    relation_pair_dict = get_relation_combinations(properties, combinations, rel_select = (0.80, 0.10))
    pairs = relation_pair_dict[combination]
    n_pairs = len(pairs)
    rel_dict
    means = []
    for prop, concept in pairs:
        means.append(get_tfidf_pair(ets, prop, concept, model_name, top_cutoff, concept_cutoff))
    if len(means) > 0:
        rel_dict['mean_tfidf'] = sum(means)/len(means)
    else:
        rel_dict['mean_tfidf'] = 0
    rel_dict['n_pairs'] = n_pairs
    if n_pairs > 3:
        n_max = 3
    else:
        n_max = n_pairs
    rel_dict['examples'] = ' '.join(['-'.join(pair) for pair in list(pairs)[:n_max]])
    rel_table.append(rel_dict)

df = pd.DataFrame(rel_table)
df.round(6)

,relation,mean_tfidf,n_pairs,examples
0,pos,0.004043,1973,used_in_cooking-pestle red-pot green-pear
1,neg,0.000979,1331,roll-windshield blue-crow lay_eggs-mangabey
2,all,0.004780,1144,used_in_cooking-pestle used_in_cooking-parer square-pantry
3,some,0.003027,829,blue-fish red-pot square-macintosh
4,few,0.000979,1331,roll-windshield blue-crow lay_eggs-mangabey
5,implied_category,0.002987,73,juicy-apple juicy-mulberry wings-weka
6,typical_of_concept,0.007391,71,used_in_cooking-tomato red-lingonberry yellow-taxi
7,typical_of_property,0.011980,4,blue-blueberry made_of_wood-bookcase hot-heat
8,affording_activity,0.008684,48,hot-spiciness warm-slipper round-cds
9,afforded_usual,0.001634,27,swim-flamingo lay_eggs-calidris lay_eggs-leatherback


In [76]:
print(df.to_latex(index=False))

\begin{tabular}{lrrl}
\toprule
            relation &  mean\_tfidf &  n\_pairs &                                                    examples \\
\midrule
                 pos &    0.004043 &     1973 &                   used\_in\_cooking-pestle red-pot green-pear \\
                 neg &    0.000979 &     1331 &                 roll-windshield blue-crow lay\_eggs-mangabey \\
                 all &    0.004780 &     1144 &  used\_in\_cooking-pestle used\_in\_cooking-parer square-pantry \\
                some &    0.003027 &      829 &                          blue-fish red-pot square-macintosh \\
                 few &    0.000979 &     1331 &                 roll-windshield blue-crow lay\_eggs-mangabey \\
    implied\_category &    0.002987 &       73 &                       juicy-apple juicy-mulberry wings-weka \\
  typical\_of\_concept &    0.007391 &       71 &          used\_in\_cooking-tomato red-lingonberry yellow-taxi \\
 typical\_of\_property &    0.011980 &        4 &        

#### Impliedness:

* implied_category should have a low mean tfidf value
* Observation: Clearly lower than mean pos value, closer to mean neg value

In [19]:
combination = ('implied_category', )

combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(ets, prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

16
0.0012937158078214462


#### Variability

* variability_limited should have a high mean tfidf value
* variability_open should have a low mean tfidf value

* Observations:


In [20]:
combination = ('variability_limited', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(ets, prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

114
0.003956634603549042


In [21]:
combination = ('variability_open', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

175
0.0017967697773990463


#### Variability in combination with impliedness should overrule impliedness

In [23]:
combination = ('implied_category', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

16
0.0012937158078214462


In [24]:
combination = ('implied_category', 'variability_limited')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(pairs)
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

{('square', 'laptop'), ('red', 'tongue'), ('sweet', 'breadfruit'), ('green', 'fenugreek'), ('made_of_wood', 'ladle'), ('fly', 'fowl'), ('made_of_wood', 'transom'), ('round', 'cabbage'), ('sweet', 'carrot'), ('round', 'onion'), ('round', 'sapodilla'), ('roll', 'cart'), ('round', 'nutlet'), ('round', 'gourd'), ('wings', 'beetle'), ('made_of_wood', 'girder'), ('round', 'lemon'), ('square', 'computer'), ('square', 'blackboard'), ('juicy', 'anjou'), ('round', 'pineapple')}
21
0.001578340098725068


In [25]:
combination = ('variability_limited', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(pairs)

{('yellow', 'nectarine'), ('made_of_wood', 'bass'), ('juicy', 'corn'), ('green', 'serviceberry'), ('roll', 'paint'), ('made_of_wood', 'roof'), ('black', 'calf'), ('yellow', 'peach'), ('blue', 'shark'), ('round', 'banjo'), ('made_of_wood', 'railing'), ('made_of_wood', 'mop'), ('red', 'ambulance'), ('round', 'pecan'), ('black', 'millipede'), ('fly', 'craft'), ('made_of_wood', 'axe'), ('juicy', 'chicken'), ('blue', 'dogbane'), ('square', 'desk'), ('juicy', 'doughnut'), ('red', 'leafs'), ('green', 'strawberry'), ('juicy', 'bean'), ('sweet', 'beetroot'), ('juicy', 'ham'), ('blue', 'starling'), ('black', 'sapsucker'), ('black', 'bear'), ('black', 'sky'), ('fly', 'capsule'), ('yellow', 'daisy'), ('black', 'penguin'), ('made_of_wood', 'beam'), ('green', 'chutney'), ('black', 'truffle'), ('red', 'lentil'), ('black', 'human'), ('red', 'carnation'), ('juicy', 'date'), ('roll', 'mandrel'), ('red', 'ant'), ('used_in_cooking', 'cake'), ('juicy', 'beet'), ('square', 'footrest'), ('juicy', 'ginger'), 

In [26]:
combination = ('implied_category', 'variability_open')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

6
0.0016937395316195085


#### Typicality

* typical_of_property should have higher values
* typical_of_concept should have lower values
* problem: they are unlikely to occur independently from one another

In [27]:
combination = ('typical_of_property', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

0


ZeroDivisionError: division by zero

In [28]:
combination = ('typical_of_concept', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

4
0.007614649891454527


#### Affordedness

In [29]:
combination = ('afforded_usual', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

3
0.00927845165257774


In [30]:
combination = ('afforded_unusual', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

20
0.00039111772001438174


In [31]:
combination = ('affording_activity', )
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

4
0.006053307639432479


#### Affordedness relations + other relations

*affording_activity*

In [32]:
combination = ('affording_activity', 'implied_category')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
print(pairs)
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

8
{('made_of_wood', 'bamboo'), ('wheels', 'dumper'), ('warm', 'tunic'), ('wheels', 'snowplow'), ('hot', 'lightning'), ('wheels', 'equipage'), ('warm', 'calfskin'), ('wheels', 'airplane')}
0.004585447563000917


In [33]:
combination = ('affording_activity', 'typical_of_concept')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

5
0.004864331820770777


In [34]:
combination = ('affording_activity', 'implied_category', 'typical_of_concept')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

57
0.0021921960452967494


*afforded_usual*

In [138]:
combination = ('afforded_usual', 'implied_category')
combinations = [set(combination)]
relation_pair_dict = get_relation_combinations(properties, combinations)
pairs = relation_pair_dict[combination]
print(len(pairs))
means = []
for prop, concept in pairs:
    means.append(get_tfidf_pair(prop, concept, model_name, top_cutoff, concept_cutoff))
print(sum(means)/len(means))

18
0.025247162010199574
